In [1]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD

df1 = pd.read_excel("data/1/Ins1_5_10_2.xlsx").fillna("none")
df1.columns = [i for i in range(1, 4)]

df2 = pd.read_excel("data/1/Ins1_5_10_2.xlsx", sheet_name="包装种类及其所需墨盒")
df2.columns = [i for i in range(1, 3)]

num_cartridges = 0 # 墨盒数
num_slots = 0 # 插槽数
num_packages = 0 # 包装数

for i, row in df1.iterrows():
    if row[1] != "none":
        num_packages += 1
    if row[2] != "none":
        num_cartridges += 1
    if row[3] != "none":
        num_slots += 1

packages = {} # 包装种类和对应墨盒需求
for i, row in df2.iterrows():
    l = row[2][1:-1].split(", ")
    tmp = [int(i) for i in l]
    packages[row[1]] = tmp

In [3]:
# 创建问题实例
model = LpProblem("Minimize", LpMinimize)

# 添加变量
x = LpVariable.dicts("cartridge_use", (range(1, num_packages+1), range(1, num_slots+1), range(1, num_cartridges+1)), cat="Binary")
y = LpVariable.dicts("switch", (range(1, num_packages), range(1, num_slots+1), range(1, num_cartridges+1)), cat="Binary")

# 目标函数
model += lpSum(y[p][s][c] for p in range(1, num_packages) for s in range(1, num_slots+1) for c in range(1, num_cartridges+1))

# 约束条件1：每个时刻插槽只能放一个墨盒
for p in range(1, num_packages+1):
    for s in range(1, num_slots+1):
        model += lpSum(x[p][s][c] for c in range(1, num_cartridges+1)) == 1

# 约束条件2：每个包装所需的所有墨盒必须放置在插槽中
for p in range(1, num_packages+1):
    for c in packages[p]:
        model += lpSum(x[p][s][c] for s in range(1, num_slots+1)) >= 1

# 约束条件3：切换约束
for p in range(1, num_packages):
    for s in range(1, num_slots+1):
        for c in range(1, num_cartridges+1):
            model += y[p][s][c] >= x[p][s][c] - x[p+1][s][c]

# 解决问题
model.solve(PULP_CBC_CMD(msg=0))

# 输出结果
total_switches = lpSum(y[p][s][c].varValue for p in range(1, num_packages) for s in range(1, num_slots+1) for c in range(1, num_cartridges+1))
print("总切换次数:", total_switches)

# 打印详细的切换信息
for p in range(1, num_packages):
    for s in range(1, num_slots+1):
        for c in range(1, num_cartridges+1):
            if y[p][s][c].varValue > 0:
                print(f"在 {p} 的插槽 {s} 中从墨盒 {c} 切换")

# # 打印墨盒的使用情况
# for p in range(1, num_packages+1):
#     for s in range(1, num_slots+1):
#         for c in range(1, num_cartridges+1):
#             if x[p][s][c].varValue > 0:
#                 print(f"包装 {p} 在插槽 {s} 中使用墨盒 {c}")


总切换次数: 5.0
在 1 的插槽 2 中从墨盒 7 切换
在 2 的插槽 1 中从墨盒 5 切换
在 3 的插槽 1 中从墨盒 4 切换
在 3 的插槽 2 中从墨盒 2 切换
在 4 的插槽 2 中从墨盒 8 切换
